# Regression

## Introduction

In this tutorial, we will be using the Theil-Sen estimator to create differentially private regressions. 

The Theil-Sen estimator is a robust method to fit a line to sample points by choosing the median of the slopes of all the lines through all the pairs of the points. Some of the advantages Theil-Sen has compared to ordinary least squares regression are that it is insensitive to outliers, significantly more accurate for skewed and heteroskedastic data, and fast algorithms for efficiently computing its parameters. It's been labled as "the most popular nonparametric technique for estimating a linear trend."

Source: [Wikipedia](https://en.wikipedia.org/wiki/Theil–Sen_estimator)

Let's get started! We will only need a few libraries for this tutorial. This tutorial has two parts: 

1. Define the Regression Functions and Test on Synthetic Data
2. Apply to Real Dataset

In [7]:
import opendp.prelude as dp
import numpy as np
import polars as pl 

dp.enable_features("honest-but-curious")
dp.assert_features("honest-but-curious")

## Defining Regression Functions

The following code defines alpha values that represent the 25th and 75th percentiles of the distribution of slopes computed from the random points in the dataset. 

Moreover, the 25th and 75th percentiles provide a nuanced understanding of the variability present in the dataset. 

### 1) Define Alphas

In [2]:
alphas = np.array([[0.25], [0.75]])

### 2) Define a Function to Compute Points on the Percentile Lines

This function partitions a dataset into random pairs, calculates a slope for each pair, and computes points on the slopes on the lines of the percentiles we specified. 

In [3]:
def f_match(data):
    #get an even number of rows
    data = np.array(data, copy=True)[:len(data) // 2 * 2]
    #shuffle the data for robustness
    np.random.shuffle(data)
    #split the data into pairs
    p1, p2 = np.array_split(data, 2)
    #compute differences
    dx, dy = (p2 - p1).T
    #compute slopes
    slope = dy / dx
    x_bar, y_bar = (p1 + p2).T / 2
    #compute points on line of percentiles
    points = slope * (alphas - x_bar) + y_bar
    #keep only pairs where the x difference is positive
    return points.T[dx > 0]


### 3) Define a Function to Compute Differentially Private Percentiles

what is k

This function creates a differentially private transformation for computing Theil-Sen percentiles by repeatedly calling the function 'f_match'. 

but f'match returns points

In [58]:
def make_theil_sen_percentiles(k: int=1):
    return dp.t.make_user_transformation(
        input_domain=dp.np_array2_domain(T=int, num_columns=2),
        input_metric=dp.symmetric_distance(),
        output_domain=dp.np_array2_domain(T=int, num_columns=2),
        output_metric=dp.symmetric_distance(),
        function=lambda x: np.vstack([f_match(x) for _ in range(int(k))]),
        stability_map=lambda b_in: k * b_in)


In [68]:
make_theil_sen_percentiles(1)

Transformation(
    input_domain   = NPArray2Domain(num_columns=2, T=i32),
    output_domain  = NPArray2Domain(num_columns=2, T=i32),
    input_metric   = SymmetricDistance(),
    output_metric  = SymmetricDistance()
)

### 4) Create a Function to Select a Specific Column

This function creates a transformation to select a specific column from a dataset. 

In [21]:
def make_select_column(j):
    return dp.t.make_user_transformation(
        input_domain=dp.np_array2_domain(T=int, num_columns=2),
        input_metric=dp.symmetric_distance(),
        output_domain=dp.vector_domain(dp.atom_domain(T=int)),
        output_metric=dp.symmetric_distance(),
        function=lambda x: x[:, j],
        stability_map=lambda b_in: b_in)


In [69]:
make_select_column(1)

Transformation(
    input_domain   = NPArray2Domain(num_columns=2, T=i32),
    output_domain  = VectorDomain(AtomDomain(T=i32)),
    input_metric   = SymmetricDistance(),
    output_metric  = SymmetricDistance()
)

### 5) Create a Function to Make Private Quantiles in Bounds

This script will need a bit of an adjustment: make_private_quantile_in_bounds should instead use the quantile scorer + RNM

In [71]:
discrete_scores = dp.t.make_quantile_score_candidates(dp.vector_domain(dp.atom_domain(T=int)), 
                                    dp.symmetric_distance(), 
                                    list(range(100)), 
                                    0.5)
discrete_scores

Transformation(
    input_domain   = VectorDomain(AtomDomain(T=i32)),
    output_domain  = VectorDomain(AtomDomain(T=usize), size=100),
    input_metric   = SymmetricDistance(),
    output_metric  = LInfDistance(T=usize)
)

In [72]:
input_space = dp.vector_domain(dp.atom_domain(T=dp.usize), 100), dp.linf_distance(T=dp.usize)

#adding noise is the measurement
select_index_measurement = dp.m.make_report_noisy_max_gumbel(*input_space, scale=1.0, optimize='min')
select_index_measurement

Measurement(
    input_domain   = VectorDomain(AtomDomain(T=usize), size=100),
    input_metric   = LInfDistance(T=usize),
    output_measure = MaxDivergence(f64))

In [73]:
private_quantiles = discrete_scores >> select_index_measurement
private_quantiles

Measurement(
    input_domain   = VectorDomain(AtomDomain(T=i32)),
    input_metric   = SymmetricDistance(),
    output_measure = MaxDivergence(f64))

### 6) Create a Differentially Private Theil-Sen Estimator

Using the functions already defined, this function privately computes the median slope, selects the appropriate column, and applies post-processing steps to compute the final regression coefficients.

In [105]:
def make_private_theil_sen(bounds, k, scale):
    # Released percentiles relate to regression coefficients by a system:
    #     p25 = .25 α + β
    #     p75 = .75 α + β
    # Equivalently:
    #     p = [[.25, 1.0], [.75, 1.0]] @ [[α], [β]]

    def postprocess(p):
        return int(-2 * np.array([[1, -1], [-.75, .25]]) @ p)

    m_median = discrete_scores >> select_index_measurement

    return make_theil_sen_percentiles(k) >> dp.c.make_basic_composition([
        make_select_column(0) >> m_median,
        make_select_column(1) >> m_median,
    ]) >> postprocess

In [ ]:
# sc_meas = dp.c.make_sequential_composition(
#     input_domain=dp.vector_domain(dp.atom_domain(T=int)),
#     input_metric=dp.symmetric_distance(),
#     output_measure=dp.max_divergence(T=float),
#     d_in=1,
#     d_mids=[2., 1.]
# )

## Creating and Applying the Mechanism to Synthetic Data

In [109]:
type(data[0][0])

numpy.int32

In [106]:
#create the mechanism
meas = make_private_theil_sen((-10, 10), k=1, scale=1.0) 
print(meas)

#generate synthetic data 
#this data is designed to simulate a linera relationship 
#noise is also added
x = np.random.normal(size=1_000)
y = 2 * x + 3 + np.random.normal(size=1_000)
data = np.stack([x, y], axis=1)
data = data.astype(np.int32)
# data = data.astype(np.float64)


print(meas(data))

Measurement(
    input_domain   = NPArray2Domain(num_columns=2, T=i32),
    input_metric   = SymmetricDistance(),
    output_measure = MaxDivergence(f64))


/var/folders/dg/nlwnz_jj7rsblr2tzwj962b40000gn/T/ipykernel_1066/4065506905.py:11: RuntimeWarning: divide by zero encountered in divide
  slope = dy / dx
/var/folders/dg/nlwnz_jj7rsblr2tzwj962b40000gn/T/ipykernel_1066/4065506905.py:11: RuntimeWarning: invalid value encountered in divide
  slope = dy / dx


OpenDPException: 
  FFI("Continued stack trace from Exception in user-defined function:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/opendp/_convert.py", line 596, in wrapper_func
    c_out = py_to_c(py_out, c_type=AnyObjectPtr, type_name=TO)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/opendp/_convert.py", line 111, in py_to_c
    return slice_as_object(value, type_name) # type: ignore[arg-type]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/opendp/_data.py", line 335, in slice_as_object
    c_raw = py_to_c(raw, c_type=FfiSlicePtr, type_name=T)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/opendp/_convert.py", line 116, i